In [24]:
import tensorflow as tf
import pandas as pd
import glob
import json
import numpy as np
import random
import os

In [25]:
def seed_it_all(seed=7):
    """ Attempt to be Reproducible """
    os.environ['PYTHONHASHSEED'] = str(seed)
    random.seed(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)

seed_it_all()

In [2]:
class CFG:
    left_ROWS_per_frame = 21
    sequence_length = 20
    batch_size = 32

In [20]:
class CustomData(tf.keras.utils.Sequence):
    def __init__(self,df,num_frames=20,batch_size=8,shuffle=True,\
                 labels_path='sign_to_prediction_index_map.json'):
        self.df = df
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.num_frames = num_frames
        self.labels  = json.load(open('sign_to_prediction_index_map.json','r'))
        self.on_epoch_end()
        
    def on_epoch_end(self):
        self.indexes = np.arange(len(self.df))
        if self.shuffle:
            np.random.shuffle(self.indexes)
    
    def __getitem__(self,index):
        batches = self.indexes[index*self.batch_size:(index+1)*self.batch_size]
        left_hand_input = np.zeros(shape=(self.batch_size,self.num_frames,CFG.left_ROWS_per_frame,3))
        right_hand_input = np.zeros(shape=(self.batch_size,self.num_frames,CFG.left_ROWS_per_frame,3))
        labels = []
        for i,row_val in enumerate(batches):
            row = self.df.iloc[row_val]
            left_hand,right_hand = self.load_video(row['path'])
            left_hand_input[i,:] = left_hand
            right_hand_input[i,:] = right_hand
            labels.append(self.labels[row['sign']])
        return [left_hand_input,right_hand_input],np.asarray(labels)
            
    def load_video(self,video_path):
        video_df = pd.read_parquet(video_path, engine='pyarrow')
        video_df.fillna(0,inplace=True)
        left_df = video_df[video_df.type=='left_hand']
        left_values = left_df[['x','y','z']].values
        left_values = left_values.reshape(-1,CFG.left_ROWS_per_frame,3)
        left_hand_array =  tf.image.resize(left_values, (CFG.sequence_length, CFG.left_ROWS_per_frame))
        right_df = video_df[video_df.type=='right_hand']
        right_values = right_df[['x','y','z']].values
        right_values = right_values.reshape(-1,CFG.left_ROWS_per_frame,3)
        right_hand_array =  tf.image.resize(right_values, (CFG.sequence_length, CFG.left_ROWS_per_frame))
        return left_hand_array, right_hand_array
    
    def __len__(self):
        return len(self.df)//self.batch_size

In [27]:
complete_df = pd.read_csv('train.csv')
from sklearn.model_selection import train_test_split

train_df, test_df = train_test_split(complete_df, test_size=0.2)

In [28]:
len(complete_df),len(train),len(test)

(94477, 75581, 18896)

In [29]:
train_datagen = CustomData(train_df,num_frames=CFG.sequence_length,batch_size=CFG.batch_size)
test_datagen = CustomData(test_df,num_frames=CFG.sequence_length,batch_size=CFG.batch_size)

In [30]:
def conv1d_lstm_block(inputs, filters):
    vector = tf.keras.layers.ConvLSTM1D(filters=32, kernel_size=3)(inputs)
    #for f in filters:
    #    vector = tf.keras.layers.Conv1D(filters=f, kernel_size=8)(vector)
    #    vector = tf.keras.layers.MaxPooling1D()(vector)
    vector = tf.keras.layers.Dropout(0.3)(vector)
    return vector

def get_model():
    input1 = tf.keras.Input((CFG.sequence_length, CFG.left_ROWS_per_frame, 3), dtype=tf.float32)
    input2 = tf.keras.Input((CFG.sequence_length, CFG.left_ROWS_per_frame, 3), dtype=tf.float32)
    left_hand_vector = conv1d_lstm_block(input1, [64])
    right_hand_vector = conv1d_lstm_block(input2, [64])
    vector = tf.keras.layers.Concatenate(axis=1)([left_hand_vector, right_hand_vector])
    vector = tf.keras.layers.Flatten()(vector)
    output = tf.keras.layers.Dense(250, activation="softmax")(vector)
    model = tf.keras.Model(inputs=[input1,input2], outputs=output)
    model.compile(tf.keras.optimizers.Adam(0.000333), "sparse_categorical_crossentropy", metrics="accuracy")
    return model

In [31]:
model = get_model()

In [32]:
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_5 (InputLayer)           [(None, 20, 21, 3)]  0           []                               
                                                                                                  
 input_6 (InputLayer)           [(None, 20, 21, 3)]  0           []                               
                                                                                                  
 conv_lstm1d_4 (ConvLSTM1D)     (None, 19, 32)       13568       ['input_5[0][0]']                
                                                                                                  
 conv_lstm1d_5 (ConvLSTM1D)     (None, 19, 32)       13568       ['input_6[0][0]']                
                                                                                            

In [34]:
file_name = "model.h5"
callbacks = [
    tf.keras.callbacks.ModelCheckpoint(
        file_name, 
        save_best_only=True, 
        restore_best_weights=True, 
        monitor="val_accuracy",
        mode="max"
    ),
    tf.keras.callbacks.EarlyStopping(
        patience=5, 
        monitor="val_accuracy",
        mode="max"
    )
]
model.fit(train_datagen,validation_data=test_datagen,\
          epochs=30, callbacks=callbacks)
model = tf.keras.models.load_model(file_name)

Epoch 1/30
2361/2361 [==============================] - 1089s 461ms/step - loss: 4.5652 - accuracy: 0.0947 - val_loss: 3.7265 - val_accuracy: 0.2066
Epoch 2/30
2361/2361 [==============================] - 1094s 463ms/step - loss: 3.4628 - accuracy: 0.2408 - val_loss: 3.1667 - val_accuracy: 0.2958
Epoch 3/30
2361/2361 [==============================] - 1136s 481ms/step - loss: 3.0669 - accuracy: 0.3074 - val_loss: 2.9344 - val_accuracy: 0.3451
Epoch 4/30
2361/2361 [==============================] - 1149s 487ms/step - loss: 2.8286 - accuracy: 0.3493 - val_loss: 2.7072 - val_accuracy: 0.3856
Epoch 5/30
2361/2361 [==============================] - 1106s 469ms/step - loss: 2.6599 - accuracy: 0.3851 - val_loss: 2.6007 - val_accuracy: 0.4039
Epoch 6/30
2361/2361 [==============================] - 1106s 469ms/step - loss: 2.5306 - accuracy: 0.4093 - val_loss: 2.4907 - val_accuracy: 0.4265
Epoch 7/30
2361/2361 [==============================] - 1126s 477ms/step - loss: 2.4259 - accuracy: 0.4306

KeyboardInterrupt: 

In [ ]:
x,y = datagen[0]

In [ ]:
video_df = pd.read_parquet('train_landmark_files/16069/100015657.parquet', engine='pyarrow')
video_df.fillna(0,inplace=True)
left_df = video_df[video_df.type=='left_hand']
left_values = left_df[['x','y','z']].values
left_values_array = left_values.reshape(-1,21,3)

In [18]:
vector = tf.image.resize(left_values_array, (22, 21))

In [33]:
Y.shape

NameError: name 'Y' is not defined